# Ablation Study

## Env

In [ ]:
# !pip install langchain_experimental
# !pip install -U langchain-community
# !pip install faiss-cpu
# !pip install --upgrade langchain langchain-openai openai
# !pip install tiktoken
# !pip install gurobipy
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
import tempfile
from langchain.agents import create_react_agent, AgentExecutor
from langchain.memory import ConversationBufferMemory
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType, Tool
from langchain.vectorstores import FAISS
from langchain.schema import Document
import openai
import time
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datetime import datetime, time
import ast
import os
from datetime import time as time_class
from io import StringIO
import sys
from langchain.schema import HumanMessage
from tqdm import tqdm


## Load Data

In [ ]:
import os
user_api_key = "YOUR_OPENAI_API_KEY"  # Replace with your OpenAI API key

def LoadFiles():
  v1 = pd.read_csv('Test_Dataset/Air_NRM/v1.csv')
  v2 = pd.read_csv('Test_Dataset/Air_NRM/v2.csv')
  demand = pd.read_csv('Test_Dataset/Air_NRM/od_demand.csv')
  flight = pd.read_csv('Test_Dataset/Air_NRM/flight.csv')
  return v1,v2,demand,flight

In [ ]:
def Classification_Agent(file_path="Large_Scale_Or_Files/RefData.csv"):
    llm1 = ChatOpenAI(
        temperature=0.0, model_name="gpt-4.1", openai_api_key=user_api_key
    )

    # Load and process the data
    loader1 = CSVLoader(file_path="Large_Scale_Or_Files/RefData.csv", encoding="utf-8")
    refdata = loader1.load()

    # Each line is a document
    refdocuments = refdata

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings(openai_api_key=user_api_key)
    vectors1 = FAISS.from_documents(refdocuments, embeddings)

    # Create a retriever
    retriever1 = vectors1.as_retriever(search_kwargs={'k': 5})

    # Create the RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm1,
        chain_type="stuff",
        retriever=retriever1,
        return_source_documents=True,
    )


    # Create a tool using the RetrievalQA chain
    qa_tool1 = Tool(
        name="FileQA",
        func=qa_chain.invoke,
        description=(
            "Use this tool to answer questions about the problem type of the text. "
            "Provide the question as input, and the tool will retrieve the relevant information from the file and use it to answer the question."
        ),
    )

    # Define few-shot examples as a string
    few_shot_examples = """

    Question: What is the problem type in operation of the text? Please give the answer directly. Text:There are three best-selling items (P1, P2, P3) on Amazon with the profit w_1,w_2,w_3.There is an independent demand stream for each of the products. The objective of the company is to decide which demands to be fufilled over a ﬁnite sales horizon [0,10] to maximize the total expected revenue from ﬁxed initial inventories. The on-hand inventories for the three items are c_1,c_2,c_3 respectively. During the sales horizon, replenishment is not allowed and there is no any in-transit inventories. Customers who want to purchase P1,P2,P3 arrive at each period accoring to a Poisson process with a_1,a_2,a_3 the arrival rates respectively. Decision variables y_1,y_2,y_3 correspond to the number of requests that the firm plans to fulfill for product 1,2,3. These variables are all positive integers.

    Thought: I need to determine the problem type of the text. I'll use the FileQA tool to retrieve the relevant information.

    Action: FileQA

    Action Input: "What is the problem type in operation of the text? text:There are three best-selling items (P1, P2, P3) on Amazon with the profit w_1, w_2, w_3. ..."

    Observation: The problem type of the text is Network Revenue Management.

    Final Answer: Network Revenue Management.

    """

    # Create the prefix and suffix for the agent's prompt
    prefix = f"""You are a helpful assistant that can answer questions about operation problems.

    Use the following examples as a guide. Always use the FileQA tool when you need to retrieve information from the file:


    {few_shot_examples}

    When you need to find information from the file, use the provided tools. And answer the question by given the answer directly. For example,

    """

    suffix = """

    Begin!

    Question: {input}
    {agent_scratchpad}"""

    agent_pc = initialize_agent(
        tools=[qa_tool1],
        llm=llm1,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        agent_kwargs={
            "prefix": prefix.format(few_shot_examples=few_shot_examples),
            "suffix": suffix,
        },
        verbose=True,
        handle_parsing_errors=True,  # Enable error handling
    )

    openai.api_request_timeout = 60 
    return agent_pc

def Problemtype(query):
    agent_pc = Classification_Agent(file_path="Large_Scale_Or_Files/RefData.csv")
    category_original=agent_pc.invoke(f"What is the problem type in operation of the text? text:{query}")
    type_output = category_original['output']
    return type_output

## Fewshot-Only

In [33]:
file_path_list = ["Test_Dataset/Air_NRM/flight.csv", "Test_Dataset/Air_NRM/od_demand.csv", "Test_Dataset/Air_NRM/v1.csv", "Test_Dataset/Air_NRM/v2.csv"]

In [34]:

import os
from typing import List, Dict

from openai import OpenAI                       # pip install openai>=1.14
from langchain.agents import Tool, initialize_agent, AgentType

# ──────────────────────────────────────────────
# 1)  Upload all CSVs once and cache their IDs
# ──────────────────────────────────────────────
client = OpenAI(api_key=user_api_key)

FILE_MAP: Dict[str, str] = {
    "flight":  "Test_Dataset/Air_NRM/flight.csv",
    "demand":  "Test_Dataset/Air_NRM/od_demand.csv",
    "v1":      "Test_Dataset/Air_NRM/v1.csv",
    "v2":      "Test_Dataset/Air_NRM/v2.csv",
}

def upload_files(paths: List[str]) -> List[str]:
    """Return a list of file-ids after uploading each path to OpenAI."""
    ids: List[str] = []
    for p in paths:
        with open(p, "rb") as f:
            ids.append(client.files.create(file=f, purpose="assistants").id)
    return ids

FILE_IDS: Dict[str, str] = {
    name: upload_files([path])[0] for name, path in FILE_MAP.items()
}

# ──────────────────────────────────────────────
# 2)  Minimal tools – they just hand back IDs
# ──────────────────────────────────────────────
def _fmt(k: str) -> str:
  return f"{k}_file_id={FILE_IDS[k]}"

csv_tool_flight = Tool(
    name="GetFlightFile",
    func=lambda _: _fmt("flight"),
    description="Returns the file-id for flight.csv so the model can load it.",
)

csv_tool_demand = Tool(
    name="GetDemandFile",
    func=lambda _: _fmt("demand"),
    description="Returns the file-id for od_demand.csv so the model can load it.",
)

csv_tool_v1v2 = Tool(
    name="GetV1V2Files",
    func=lambda _: f"v1_file_id={FILE_IDS['v1']}, v2_file_id={FILE_IDS['v2']}",
    description="Returns the file-ids for v1.csv and v2.csv.",
)

TOOLS = [csv_tool_flight, csv_tool_demand, csv_tool_v1v2]


In [35]:

# ──────────────────────────────────────────────
# 3)  Few-shot prompt pieces
# ──────────────────────────────────────────────

example_data_description_NP = "avg_price={{'(AB,11:20,f)': '1140.3', '(AB,11:20,l)': '429.26'}} \n value_list ={{'(CA,7:40,f)': 0.228917, '(CA,7:40,l)': 0.54522, '(CB,7:40,f)': 0, '(CB,7:40,l)': 0, '(BA,9:05,f)': 1.043622, '(BA,9:05,l)': 0.0625, '(BC,9:05,f)': 0, '(BC,9:05,l)': 0, '(CB,10:45,f)': 0, '(CB,10:45,l)': 0, '(AB,11:20,f)': 0.342687, '(AB,11:20,l)': 0.24677, '(BC,11:20,f)': 0, '(BC,11:20,l)': 0}}\n ratio_list={{'(CA,7:40,f)': 1.0, '(CA,7:40,l)': 0.97, '(CB,7:40,f)': 0, '(CB,7:40,l)': 0, '(BA,9:05,f)': 1.0, '(BA,9:05,l)': 0.29, '(BC,9:05,f)': 0, '(BC,9:05,l)': 0, '(CB,10:45,f)': 0, '(CB,10:45,l)': 0, '(AB,11:20,f)': 0.67, '(AB,11:20,l)': 1.0, '(BC,11:20,f)': 0, '(BC,11:20,l)': 0}}\nvalue_0_list={{'CA': 1, 'CB': 0, 'BA': 1, 'BC': 0, 'AB': 1}}\n ratio_0_list={{'CA': 1.4, 'CB': 0, 'BA': 1.53, 'BC': 0, 'AB': 1.41}}\navg_pax={{'CA': '4807.43', 'CB': '4807.43', 'BC': '38965.86', 'AB': '38965.86', 'BA': '33210.71'}}\ncapacity_consum=1.2\noption_num=None\nsigma_inflow_A=['(CA,7:40)', '(BA,9:05)']\nsigma_outflow_A=['(AB,11:20)']\nsigma_inflow_B=['(CB,7:40)', '(CB,10:45)', '(AB,11:20)']\nsigma_outflow_B=['(BA,9:05)', '(BC,9:05)', '(BC,11:20)']\nsigma_inflow_C=['(BC,9:05)', '(BC,11:20)']\nsigma_outflow_C=['(CA,7:40)', '(CB,7:40)', '(CB,10:45)']\nM = 10000000\nflight_capacity=187\n"

fewshot_only_NP = f"""
Question: Based on flight ticket options provided in file ""./Test_Dataset/Air_NRM/information.csv"", along with their average passengers (Avg Pax), average prices (Avg Price), and capacity coefficients (Flex Cpy Coef), considering that each Eco_flex ticket consumes 2 units of flight capacity and each Eco_lite ticket consumes 1 unit of capacity, while enforcing flow conservation constraints at each airport for long-term planning model, develop a Sales-Based Linear Programming (SBLP) model. 

The goal of this model is to recommend the optimal 3 flights that maximize total ticket sale revenue, specifically among flights where the origin-destination pairs are: 
OD = ('A', 'B') or OD = ('A', 'C') or OD = ('B', 'A') or OD = ('B', 'C') or ('C', 'A') or ('C', 'B') in a departure period (7am-12:00am) in which the optional flights are:
[(OD = ('C', 'A') AND Departure Time='7:40'),
(OD = ('C', 'B') AND Departure Time='7:40'),
(OD = ('B', 'A') AND Departure Time='9:05'),
(OD = ('B', 'C') AND Departure Time='9:05'),
(OD = ('C', 'B') AND Departure Time='10:45'),
(OD = ('A', 'B') AND Departure Time='11:20'),
(OD = ('B', 'C') AND Departure Time='11:20')]

Thought: First, I have known the capacity_consum = 2, option_num = 3, sigma_inflow_A=['(CA,7:40)', '(BA,9:05)'], sigma_outflow_A=['(AB,11:20)'], sigma_inflow_B=['(CB,7:40)', '(CB,10:45)', '(AB,11:20)'], sigma_outflow_B=['(BA,9:05)', '(BC,9:05)', '(BC,11:20)']\nsigma_inflow_C=['(BC,9:05)', '(BC,11:20)'], sigma_outflow_C=['(CA,7:40)', '(CB,7:40)', '(CB,10:45)'], M = 10000000, flight_capacity=187. Then, I need to have access to flight.csv for the model.
Action:csv_tool_flight
Action Input: Returns the file-id for flight.csv so the model can load it.
Observation:
flight_file_id=file-5fy2hgfr7YWZJ6AzmifHhj
Thought: With the file, I can get the flight data for the flight options and organize them in dicts.
Observation:
avg_price={{'(AB,11:20,f)': '1140.3', '(AB,11:20,l)': '429.26'}}
Thought: Now I need to have access to od_demand.csv for the model.
Action:csv_tool_demand
Action Input: Returns the file-id for od_demand.csv so the model can load it.
Observation:
demand_file_id=file-8dY7Ug4zdwi5xvC3dfbLRy
Thought: With the file, I can get the demand data for the flight options and organize them in dicts.
Observation:
avg_pax={{'CA': '4807.43', 'CB': '4807.43', 'BC': '38965.86', 'AB': '38965.86', 'BA': '33210.71'}}
Thought: Now I need to have access to v1.csv and v2.csv for the model.
Action:csv_tool_v1v2
Action Input: Returns the file-ids for v1.csv and v2.csv.
Observation:
v1_file_id=file-XvUswUf66ri7i1g561z43J, v2_file_id=file-26WPsKutxJnXPp7uMK8uWC
Thought: With the file, I can get the attraction values and shadow attraction ratios for the flight options and organize them in dicts.

Obseravtion:
value_list ={{'(CA,7:40,f)': 0.228917, '(CA,7:40,l)': 0.54522, '(CB,7:40,f)': 0, '(CB,7:40,l)': 0, '(BA,9:05,f)': 1.043622, '(BA,9:05,l)': 0.0625, '(BC,9:05,f)': 0, '(BC,9:05,l)': 0, '(CB,10:45,f)': 0, '(CB,10:45,l)': 0, '(AB,11:20,f)': 0.342687, '(AB,11:20,l)': 0.24677, '(BC,11:20,f)': 0, '(BC,11:20,l)': 0}}

ratio_list={{'(CA,7:40,f)': 1.0, '(CA,7:40,l)': 0.97, '(CB,7:40,f)': 0, '(CB,7:40,l)': 0, '(BA,9:05,f)': 1.0, '(BA,9:05,l)': 0.29, '(BC,9:05,f)': 0, '(BC,9:05,l)': 0, '(CB,10:45,f)': 0, '(CB,10:45,l)': 0, '(AB,11:20,f)': 0.67, '(AB,11:20,l)': 1.0, '(BC,11:20,f)': 0, '(BC,11:20,l)': 0}}

value_0_list={{'CA': 1, 'CB': 0, 'BA': 1, 'BC': 0, 'AB': 1}}

ratio_0_list={{'CA': 1.4, 'CB': 0, 'BA': 1.53, 'BC': 0, 'AB': 1.41}}

Thought: Now I know the final answer.
Final Answer:
### Objective Function

\[
\max \quad \sum_(l,k,j) avg\_price[(l,k,j)] \cdot x[(l,k,j)]
\]

### Constraints

#### 1. Capacity Constraints

For each flight (OD pair \( l \), departure time \( k \)):

\[
capacity\_consum \cdot x[(l,k,f)] + x[(l,k,l)] \leq flight\_capacity
\]

#### 2. Balance Constraints

For each OD pair \( l \):

\[
ratio\_0\_list[l] \cdot x_o[l] + \sum_k \sum_j ratio\_list[(l,k,j)] \cdot x[(l,k,j)] = avg\_pax[l]
\]

#### 3. Scale Constraints

For each ticket option \((l,k,j)\):

\[
x[(l,k,j)]/value\_list[(l,k,j)] - x_o[l]/value\_0\_list[l] \leq 0
\]

#### 4. Nonnegativity Constraints

\[
x[(l,k,j)] \geq 0, \quad x_o[l] \geq 0
\]
- x = avg_price.keys()
- x_o = avg_pax.keys()

### Retrieved Information
{example_data_description_NP}

### Generated Code

```python
import gurobipy as gp
from gurobipy import GRB
{example_data_description_NP}

model = gp.Model("sales_based_lp")
x = model.addVars(avg_price.keys(), lb=0, name="x")
x_o = model.addVars(value_0_list.keys(), lb=0, name="x_o")

model.setObjective(gp.quicksum(avg_price[key] * x[key] for key in avg_price.keys()), GRB.MAXIMIZE)

paired_keys = []
for i in range(0, len(avg_price.keys()), 2):
    if i + 1 < len(avg_price.keys()):
        names = list(avg_price.keys())
        model.addConstr(
            capacity_consum* x[names[i]] + x[names[i + 1]] <= 187,
            name=f"capacity_constraint"
        )

for l in ratio_0_list.keys():
    temp = 0
    for key in ratio_list.keys():
        if l in key:
            temp += ratio_list[key] * x[key]
    model.addConstr(
        float(ratio_0_list[l]) * x_o[l] + temp == float(avg_pax[l])
    )

for i in value_list.keys():
    for l in ratio_0_list.keys():
        if l in i:
            model.addConstr(
                value_0_list[l]  * x[i] <= value_list[i] * x_o[l]
            )

model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(v.varName, v.x)
    print("Optimal objective value:", model.objVal)
else:
    print("No optimal solution found.")

Note: Please retrieve all neccessary information from the CSV file to generate the answer. When you generate the answer, please output required parameters in a whole text, including all vectors and matrices.
"""


In [36]:
example_data_description_CA = "avg_price={{'(BA,20:25,f)': '1133.9', '(BA,20:25,l)': '410.31'}} \n value_list ={{'(BA,12:25,f)': 0.177309, '(BA,12:25,l)': 0.471942, '(CB,14:15,f)': 0, '(CB,14:15,l)': 0, '(BA,20:25,f)': 0.780656, '(BA,20:25,l)': 1.332733}}\n ratio_list={{'(BA,12:25,f)': 1.0, '(BA,12:25,l)': 0.0, '(CB,14:15,f)': 0, '(CB,14:15,l)': 0, '(BA,20:25,f)': 0.74, '(BA,20:25,l)': 0.97}}\nvalue_0_list={{'BA': 1, 'CB': 0}}\n ratio_0_list={{'BA': 1.53, 'CB': 0}}\navg_pax={{'BA': '33210.71', 'CB': '4807.43'}}\ncapacity_consum = 1.2\nflight_capacity = 187 \n"

fewshot_only_CA = f"""
Question: Based on all flight ticket choices in 'information1_addB_2.csv' with attraction values in v1 and shadow attraction value ratios in v2, develop the SBLP(sales-based linear programming) formulation  with flights (OD = ('B', 'A') AND Departure Time='12:25'), (OD = ('C', 'B') AND Departure Time='14:15'), (OD = ('B', 'A') AND Departure Time='20:25') that maximize the total revenue of flight ticket sales. The SBLP should include decision variables, objective function, balance constraints, scale constraints, nonnegative constraints.

Thought: First, I have known the capacity_consum = 1.2, flight_capacity = 187. Then, I need to have access to flight.csv for the model.
Action:csv_tool_flight
Action Input: Returns the file-id for flight.csv so the model can load it.
Observation:
flight_file_id=file-5fy2hgfr7YWZJ6AzmifHhj
Thought: With the file, I can get the flight data for the flight options and organize them in dicts.
Observation:
avg_price={{'(BA,20:25,f)': '1133.9', '(BA,20:25,l)': '410.31'}}
Thought: Now I need to have access to od_demand.csv for the model.
Action:csv_tool_demand
Action Input: Returns the file-id for od_demand.csv so the model can load it.
Observation:
demand_file_id=file-8dY7Ug4zdwi5xvC3dfbLRy
Thought: With the file, I can get the demand data for the flight options and organize them in dicts.
Observation:
avg_pax={{'BA': '33210.71', 'CB': '4807.43'}}
Thought: Now I need to have access to v1.csv and v2.csv for the model.
Action:csv_tool_v1v2
Action Input: Returns the file-ids for v1.csv and v2.csv.
Observation:
v1_file_id=file-XvUswUf66ri7i1g561z43J, v2_file_id=file-26WPsKutxJnXPp7uMK8uWC
Thought: With the file, I can get the attraction values and shadow attraction ratios for the flight options and organize them in dicts.

Obseravtion:
value_list ={{'(BA,12:25,f)': 0.177309, '(BA,12:25,l)': 0.471942, '(CB,14:15,f)': 0, '(CB,14:15,l)': 0, '(BA,20:25,f)': 0.780656, '(BA,20:25,l)': 1.332733}}
ratio_list={{'(BA,12:25,f)': 1.0, '(BA,12:25,l)': 0.0, '(CB,14:15,f)': 0, '(CB,14:15,l)': 0, '(BA,20:25,f)': 0.74, '(BA,20:25,l)': 0.97}}
value_0_list={{'BA': 1, 'CB': 0}}
ratio_0_list={{'BA': 1.53, 'CB': 0}}


Thought: Now I know the final answer.
Final Answer:
### Objective Function

\[
\max \quad \sum_(l,k,j) avg\_price[(l,k,j)] \cdot x[(l,k,j)]
\]

### Constraints

#### 1. Capacity Constraints

For each flight (OD pair \( l \), departure time \( k \)):

\[
capacity\_consum \cdot x[(l,k,f)] + x[(l,k,l)] \leq flight\_capacity
\]

#### 2. Balance Constraints

For each OD pair \( l \):

\[
ratio\_0\_list[l] \cdot x_o[l] + \sum_k \sum_j ratio\_list[(l,k,j)] \cdot x[(l,k,j)] = avg\_pax[l]
\]

#### 3. Scale Constraints

For each ticket option \((l,k,j)\):

\[
x[(l,k,j)]/value\_list[(l,k,j)] - x_o[l]/value\_0\_list[l] \leq 0
\]

#### 4. Nonnegativity Constraints

\[
x[(l,k,j)] \geq 0, \quad x_o[l] \geq 0
\]
- x = avg_price.keys()
- x_o = avg_pax.keys()

### Retrieved Information
{example_data_description_CA}

### Generated Code

```python
import gurobipy as gp
from gurobipy import GRB
{example_data_description_CA}

model = gp.Model("sales_based_lp")
x = model.addVars(avg_price.keys(), lb=0, name="x")
x_o = model.addVars(value_0_list.keys(), lb=0, name="x_o")

model.setObjective(gp.quicksum(avg_price[key] * x[key] for key in avg_price.keys()), GRB.MAXIMIZE)

paired_keys = []
for i in range(0, len(avg_price.keys()), 2):
    if i + 1 < len(avg_price.keys()):
        names = list(avg_price.keys())
        model.addConstr(
            capacity_consum* x[names[i]] + x[names[i + 1]] <= 187,
            name=f"capacity_constraint"
        )

for l in ratio_0_list.keys():
    temp = 0
    for key in ratio_list.keys():
        if l in key:
            temp += ratio_list[key] * x[key]
    model.addConstr(
        float(ratio_0_list[l]) * x_o[l] + temp == float(avg_pax[l])
    )

for i in value_list.keys():
    for l in ratio_0_list.keys():
        if l in i:
            model.addConstr(
                value_0_list[l]  * x[i] <= value_list[i] * x_o[l]
            )

model.optimize()

if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for v in model.getVars():
        print(v.varName, v.x)
    print("Optimal objective value:", model.objVal)
else:
    print("No optimal solution found.")

Note: Please retrieve all neccessary information from the CSV file to generate the answer. When you generate the answer, please output required parameters in a whole text, including all vectors and matrices.
"""


In [37]:
file_id_listing = "\n".join(f"{k}: {v}" for k, v in FILE_IDS.items())
escaped_fewshot_CA = fewshot_only_CA.replace("{", "{{").replace("}", "}}")
escaped_fewshot_NP = fewshot_only_NP.replace("{", "{{").replace("}", "}}")

PREFIX_CA = f"""
You are an assistant that generates a SBLP mathematical model and the corresponding Gurobi Python code based on the user's description and provided CSV data.  When you need to retrieve information from the CSV file, please use the GetFlightFile tool, GetDemandFile tool and GetV1V2Files tool. Please refer to the following example and generate the answer in the same format:

{escaped_fewshot_CA}

Note: Please retrieve all neccessary information from the CSV file to generate the answer. When you generate the answer, please output required parameters in a whole text, including all vectors and matrices.

"""

PREFIX_NP = f"""
You are an assistant that generates a SBLP mathematical model and the corresponding Gurobi Python code based on the user's description and provided CSV data.  When you need to retrieve information from the CSV file, please use the GetFlightFile tool, GetDemandFile tool and GetV1V2Files tool. Please refer to the following example and generate the answer in the same format:

{escaped_fewshot_NP}

Note: Please retrieve all neccessary information from the CSV file to generate the answer. When you generate the answer, please output required parameters in a whole text, including all vectors and matrices.

"""


SUFFIX = """

Begin!

User Description: {input}
{agent_scratchpad}
"""

In [38]:
def Fewshot_Only_CA(query: str) -> str:

    llm = ChatOpenAI(temperature=0.0, model_name='gpt-4.1',top_p = 1,n = 1, openai_api_key=user_api_key)

    agent_fewshot = initialize_agent(
    tools=TOOLS,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": PREFIX_CA, "suffix": SUFFIX},
    verbose=True,
    handle_parsing_errors=True,
    )

    result = agent_fewshot.invoke({"input": query})
    output_model = result['output']

    return output_model


def Fewshot_Only_NP(query: str) -> str:

    llm = ChatOpenAI(temperature=0.0, model_name='gpt-4.1',top_p = 1,n = 1, openai_api_key=user_api_key)

    agent_fewshot = initialize_agent(
    tools=TOOLS,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_kwargs={"prefix": PREFIX_NP, "suffix": SUFFIX},
    verbose=True,
    handle_parsing_errors=True,
    )

    result = agent_fewshot.invoke({"input": query})
    output_model = result['output']

    return output_model


In [39]:
import time

def Process_Input(query):
  category_original = Problemtype(query)
  print(f"Problem type classification finished, it belongs to {category_original}.")
  if "Sales-Based Linear Programming" in    category_original:
    print("Processing AirNRM queries")
    if "flow conservation constraints" in query or "flow conservation constraint" in query:
        print('----------Flow Constraints----------')
        print("Recommend Optimal Flights With Flow Conervation Constraints")
        output_model= Fewshot_Only_NP(query)
        Type = "Policy_Flow"

    else:
        print('----------CA----------')
        print("Only Develop Mathematic Formulations. No Recommendation for Flights.")
        output_model = Fewshot_Only_CA(query)
        Type = "CA"
  return Type,output_model


from tqdm import tqdm

def Batch_Process_Queries(df, query_column='Query'):
    """Process in Batches"""
    results = []

    for query in tqdm(df['Query'], desc="Processing Queries"):
        category, output_model = Process_Input(query)
        record = {
            "Category": category,
            "Original_Query": query,
            "Output": output_model,
        }
        results.append(record)
        time.sleep(10)  # Optional: to avoid hitting API rate limits

    return pd.DataFrame(results)



In [40]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import re
import numpy as np

def extract_objective(log_text):

    if "Optimal objective" in log_text:
        pattern = r"Optimal objective value: ([-+]?\d*\.?\d+)"
        match = re.search(pattern, log_text)
        if match:
            return float(match.group(1))
    elif "Best objective" in log_text:
        pattern = r"Best objective\s+([-+]?\d*\.?\d+e[-+]?\d+)"
        match = re.search(pattern, log_text, re.IGNORECASE)
        if match:
            return float(match.group(1))
    return None



def gain_obj(df_out):
    output_code = df_out['Code'].tolist()
    obj = []
    i = 1
  
    for code in output_code:
        print(i)
      
        namespace = {'gp': gp, 'GRB': GRB}

      
        log_output = []

  
        class LogCapture:
            def write(self, message):
                log_output.append(message)

            def flush(self):  
                pass

        import sys
        log_capture = LogCapture()
        original_stdout = sys.stdout 
        sys.stdout = log_capture 

        try:
            exec(code, namespace)
        except Exception as e:
            log_output.append(f"Error: {e}\n")
            print(f"Error executing code block {i}: {e}")
        finally:
            sys.stdout = original_stdout  

   
        log_text = ''.join(log_output)

  
        optimal_value = extract_objective(log_text)
        if optimal_value is not None:
            obj.append(optimal_value)
            print(f"Optimal Value: {optimal_value}")
        else:

            obj.append("No optimal value found.")
            print("No optimal value found.")
        i += 1
    return obj


In [41]:
df = pd.read_csv('Test_Dataset/Air_NRM/query_CA.csv')
result_df = Batch_Process_Queries(df)
result_df.to_csv("FewshotOnly_CA_test_GPT4.1_New.csv", index=False)

Code = []
for i in range(len(result_df['Output'])):
    text = result_df['Output'][i]
    pattern = r'```python(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    code = matches[0]
    if 'gurobipy' in code:
        Code.append(code)
    else:
        if len(matches) > 0:
            Code.append(matches[1])
        else:
            Code.append("No code found")

code_df = pd.DataFrame(Code, columns=['Code'])
code_df.to_csv("FewshotOnly_code_CA_test_GPT4.1_New.csv", index=False)

obj = gain_obj(code_df)
obj_df = pd.DataFrame({'Optimal Value': obj})
obj_df.to_csv('OBJ_CA_4.1_FewshotOnly.csv', index=False)

combined_df = pd.concat([result_df, code_df,obj_df], axis=1)
combined_df.to_csv("final_answer_CA_test_GPT4.1_FewshotOnly.csv", index=False)

Processing Queries:   0%|          | 0/15 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Question: What is the problem type in operation of the text? text:Based on all flight ticket choices in 'flight.csv' and 'od_demand.csv' with attraction values in v1 and shadow attraction value ratios in v2, develop the SBLP(sales-based linear programming) formulation for among flights (OD = ('A', 'C') AND Departure Time='23:00'), (OD = ('A', 'C') AND Departure Time='19:05'), (OD = ('B', 'A') AND Departure Time='15:40'), (OD = ('B', 'A') AND Departure Time='18:50'), (OD = ('C', 'A') AND Departure Time='16:55'), (OD = ('B', 'A') AND Departure Time='09:05'), (OD = ('C', 'A') AND Departure Time='07:40'),  that maximize the total revenue of flight ticket sales. The SBLP should include decision variables, objective function, balance constraints, scale constraints, nonnegative constraints and selection constraints. Please consider that each Eco_flex ticket consumes 2 units of flight capacity and each Eco_lite ticket consumes 1 unit of capacity

Thought

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


Final Answer: Sales-Based Linear Programming-Flow

> Finished chain.
Problem type classification finished, it belongs to Sales-Based Linear Programming-Flow.
Processing AirNRM queries
----------CA----------
Only Develop Mathematic Formulations. No Recommendation for Flights.


> Entering new AgentExecutor chain...
Question: Based on all flight ticket choices in 'flight.csv' and 'od_demand.csv' with attraction values in v1 and shadow attraction value ratios in v2, develop the SBLP(sales-based linear programming) formulation for among flights (OD = ('A', 'C') AND Departure Time='23:00'), (OD = ('A', 'C') AND Departure Time='19:05'), (OD = ('B', 'A') AND Departure Time='15:40'), (OD = ('B', 'A') AND Departure Time='18:50'), (OD = ('C', 'A') AND Departure Time='16:55'), (OD = ('B', 'A') AND Departure Time='09:05'), (OD = ('C', 'A') AND Departure Time='07:40'),  that maximize the total revenue of flight ticket sales. The SBLP should include decision variables, objective function, balance co

Processing Queries:   7%|▋         | 1/15 [01:17<18:10, 77.91s/it]

...



> Entering new AgentExecutor chain...
Question: What is the problem type in operation of the text? text:Based on all flight ticket choices in 'flight.csv' and 'od_demand.csv' with attraction values in v1 and shadow attraction value ratios in v2, develop the SBLP(sales-based linear programming) formulation for among flights (OD = ('B', 'A') AND Departure Time='15:40'), (OD = ('B', 'A') AND Departure Time='20:25'), (OD = ('A', 'C') AND Departure Time='19:05'), (OD = ('A', 'C') AND Departure Time='23:00'), (OD = ('B', 'A') AND Departure Time='09:05'), (OD = ('B', 'A') AND Departure Time='06:25'), (OD = ('A', 'B') AND Departure Time='22:45'), (OD = ('A', 'B') AND Departure Time='17:05'), (OD = ('B', 'A') AND Departure Time='18:50'), (OD = ('A', 'B') AND Departure Time='06:40'), (OD = ('A', 'B') AND Departure Time='11:20'), (OD = ('C', 'A') AND Departure Time='07:40'), (OD = ('A', 'C') AND Departure Time='17:25'), (OD = ('A', 'B') AND Departure Time='07:55'), (OD = ('C', 'A') AND Departur

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


I now know the final answer
Final Answer: Sales-Based Linear Programming-Flow (with Selection Constraints)

> Finished chain.
Problem type classification finished, it belongs to Sales-Based Linear Programming-Flow (with Selection Constraints).
Processing AirNRM queries
----------CA----------
Only Develop Mathematic Formulations. No Recommendation for Flights.


> Entering new AgentExecutor chain...
Question: Based on all flight ticket choices in 'flight.csv' and 'od_demand.csv' with attraction values in v1 and shadow attraction value ratios in v2, develop the SBLP(sales-based linear programming) formulation for among flights (OD = ('B', 'A') AND Departure Time='15:40'), (OD = ('B', 'A') AND Departure Time='20:25'), (OD = ('A', 'C') AND Departure Time='19:05'), (OD = ('A', 'C') AND Departure Time='23:00'), (OD = ('B', 'A') AND Departure Time='09:05'), (OD = ('B', 'A') AND Departure Time='06:25'), (OD = ('A', 'B') AND Departure Time='22:45'), (OD = ('A', 'B') AND Departure Time='17:05'),

Processing Queries: 100%|██████████| 15/15 [19:32<00:00, 78.17s/it]


IndexError: list index out of range

In [43]:
df = pd.read_csv('Test_Dataset/Air_NRM/query_NP_Flow.csv')
result_df = Batch_Process_Queries(df)
result_df.to_csv("FewshotOnly_NP_test_GPT4.1_New.csv", index=False)

Code = []
for i in range(len(result_df['Output'])):
    text = result_df['Output'][i]
    pattern = r'```python(.*?)```'
    matches = re.findall(pattern, text, re.DOTALL)
    code = matches[0]
    if 'gurobipy' in code:
        Code.append(code)
    else:
        if len(matches) > 0:
            Code.append(matches[1])
        else:
            Code.append("No code found")

code_df = pd.DataFrame(Code, columns=['Code'])
code_df.to_csv("FewshotOnly_code_NP_test_GPT4.1_New.csv", index=False)

obj = gain_obj(code_df)
obj_df = pd.DataFrame({'Optimal Value': obj})
obj_df.to_csv('OBJ_NP_4.1_FewshotOnly.csv', index=False)

combined_df = pd.concat([result_df, code_df,obj_df], axis=1)
combined_df.to_csv("final_answer_NP_test_GPT4.1_FewshotOnly.csv", index=False)

Processing Queries:   0%|          | 0/15 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
Question: What is the problem type in operation of the text? text:Based on flight ticket options provided in the table, along with their average passengers (Avg Pax), average prices (Avg Price), and capacity coefficients (Flex Cpy Coef), considering that each Eco_flex ticket consumes 1.2 units of flight capacity and each Eco_lite ticket consumes 1 unit of capacity, while enforcing flow conservation constraints at each airport for long-term planning model, develop a Sales-Based Linear Programming (SBLP) model. 

The goal of this model is to recommend the optimal 2 flights that maximize total ticket sale revenue, specifically among flights where the origin-destination pairs are: 
OD = ('A', 'B') or OD = ('A', 'C') or OD = ('B', 'A') or ('C', 'A') in a departure period (05:00-12:00) in which the optional flights are: 
(OD = ('A', 'B') AND Departure Time='11:20'), (OD = ('A', 'B') AND Departure Time='06:40'), (OD = ('A', 'B') AND Departure Time='07:5

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


I now know the final answer
Final Answer: Sales-Based Linear Programming with Flow Conservation Constraints (SBLP-Flow)

> Finished chain.
Problem type classification finished, it belongs to Sales-Based Linear Programming with Flow Conservation Constraints (SBLP-Flow).
Processing AirNRM queries
----------Flow Constraints----------
Recommend Optimal Flights With Flow Conervation Constraints


> Entering new AgentExecutor chain...
Question: Based on flight ticket options provided in the table, along with their average passengers (Avg Pax), average prices (Avg Price), and capacity coefficients (Flex Cpy Coef), considering that each Eco_flex ticket consumes 1.2 units of flight capacity and each Eco_lite ticket consumes 1 unit of capacity, while enforcing flow conservation constraints at each airport for long-term planning model, develop a Sales-Based Linear Programming (SBLP) model. 

The goal of this model is to recommend the optimal 2 flights that maximize total ticket sale revenue, spec

Processing Queries:   7%|▋         | 1/15 [01:45<24:36, 105.46s/it]

...



> Entering new AgentExecutor chain...
Question: What is the problem type in operation of the text? text:Based on flight ticket options provided in the table, along with their average passengers (Avg Pax), average prices (Avg Price), and capacity coefficients (Flex Cpy Coef), considering that each Eco_flex ticket consumes 3.2 units of flight capacity and each Eco_lite ticket consumes 1 unit of capacity, while enforcing flow conservation constraints at each airport for long-term planning model, develop a Sales-Based Linear Programming (SBLP) model. 

The goal of this model is to recommend the optimal 20 flights that maximize total ticket sale revenue, specifically among flights where the origin-destination pairs are: 
OD = ('A', 'B') or OD = ('A', 'C') or OD = ('B', 'A') or ('C', 'A') in a departure period (00:00-23:00) in which the optional flights are: 
(OD = ('A', 'B') AND Departure Time='11:20'), (OD = ('A', 'B') AND Departure Time='17:05'), (OD = ('A', 'B') AND Departure Time='19:

WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.


I now know the final answer
Final Answer: Sales-Based Linear Programming with Flow Conservation Constraints (SBLP-Flow)

> Finished chain.
Problem type classification finished, it belongs to Sales-Based Linear Programming with Flow Conservation Constraints (SBLP-Flow).
Processing AirNRM queries
----------Flow Constraints----------
Recommend Optimal Flights With Flow Conervation Constraints


> Entering new AgentExecutor chain...
Question: Based on flight ticket options provided in the table, along with their average passengers (Avg Pax), average prices (Avg Price), and capacity coefficients (Flex Cpy Coef), considering that each Eco_flex ticket consumes 3.2 units of flight capacity and each Eco_lite ticket consumes 1 unit of capacity, while enforcing flow conservation constraints at each airport for long-term planning model, develop a Sales-Based Linear Programming (SBLP) model. 

The goal of this model is to recommend the optimal 20 flights that maximize total ticket sale revenue, spe

Processing Queries: 100%|██████████| 15/15 [21:24<00:00, 85.62s/it]

1
No optimal value found.
2
No optimal value found.
3
No optimal value found.
4
No optimal value found.
5
No optimal value found.
6
No optimal value found.
7
No optimal value found.
8
No optimal value found.
9
No optimal value found.
10
No optimal value found.
11
No optimal value found.
12
No optimal value found.
13
No optimal value found.
14
No optimal value found.
15
No optimal value found.
